In [ ]:
'''8. Generator:Celem generatora było zapełnienie bazy jak najbardziej prawdopodobnymi danymi. 
    Wszystkie zwracane krotki są zgodne z założonymi warunkami integralnościowymi.
    Są też pewne losowe zdarzenia, generowane z określonym prawdopodobieństwem 
    (np. większość rezerwacji jest opłacona w całości, jednak co jakiś czas zdarza się nadpłata 
     albo brak płatności.Wielkość przykładowych danych:
     Participants: 2900
     Customers: 100
     Conferences: 72
     Days: 136
     Workshops: 330
     Prices: 452
     ConferencesBookings: 186
     DaysBookings: 359
     DaysEnrolment: 6312
     WorkshopsBookings: 849
     WorkshopsEnrolment: 8117
     Payments: 435
     Generowanie zajmuje zazwyczaj kilka sekund.
     Parsowanie pliku z danymi (od fakenamegenerator.com) do data.txt:
'''

In [190]:

import csv

with open('fakeName.csv', newline='', encoding='utf-8-sig') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    data = [[row[1], row[2], row[8], row[9] , row[11],row[3],row[4],row[7], row[6]] for row in spamreader]
    output = dict()
    output['first_name'] = [row[0] for row in data][1:]
    output['surname'] = [row[1] for row in data][1:]
    output['address'] = [row[5] for row in data][1:]
    output['city'] = [row[6] for row in data][1:]
    output['country'] = [row[7] for row in data][1:]
    output['mail'] = [row[2] for row in data][1:]
    output['phone'] = [row[3] for row in data][1:]
    output['companies'] = [row[4] for row in data][1:]
    output['postal_code'] = [row[8] for row in data][1:]
    #output['conferencenames'] = [row[6] for row in data]
    
    with open('data.txt', 'w+') as f:
        f.write(str(output))




In [405]:
#Generacja danych:


import datetime
import random
import math

with open('data.txt', 'r') as f:
    data = eval(f.read())
list_of_participants = list()

list_of_clients = list()
list_of_clients_to_destroy = list()

list_of_conferences = list()

list_of_conferencesPlaces = list()

list_of_customers = list()
list_of_conferences_bookings = list()

list_of_discounts = list()


list_of_days = list()
list_of_days_bookings = list()
list_of_days_enrollment = list()

list_of_companies = list()

list_of_people = list()
list_of_prices = list()
list_of_payments = list()

list_of_students  = list()

list_of_workshops = list()
list_of_workshops_days = list()

list_of_workshop_bookings = list()
list_of_workshop_enrollment = list()



In [400]:

def workshops_participation():
    cols = '(Person_ID, Workshop_Reservation_ID)'
    q_begin = 'INSERT INTO [dbo].[Workshops_Participant] ' + cols + ' VALUES '
    q = q_begin
    counter = 1
    
    for i, wb in enumerate(list_of_workshop_bookings):
        workshop_booking_id = wb[0]
        day_booking_id = wb[1]
        workshop_id = wb[2]
        places = wb[4]
        start_time = list_of_workshops_days[workshop_id - 1][5]
        end_time = list_of_workshops_days[workshop_id - 1][6]
        de = [dr[0] for dr in list_of_conferences_bookings if dr[6] == day_booking_id]
        
        
        allowed_de = set(de)
        
        for deID in de:
            previous_workshops = [list_of_workshop_bookings[pe[2]-1][2]
                                  for pe in list_of_workshop_enrollment if pe[1] == deID]
            

            previous_times = [list_of_workshops_days[workshopID - 1][5:7] for workshopID in previous_workshops]
            
            for time in previous_times:
                if not (end_time < time[0] or time[1] < start_time):
                    allowed_de.remove(deID)
                    break
                    
        if random.randint(0, 8) == 0:
            number = random.randint(0, len(allowed_de))
        else:
            number = len(allowed_de)
           
        
        booked = random.sample(allowed_de, int(min(places, number)))
        
        
        for j, deID in enumerate(booked):
            q = q + '(' + str(deID) + ', '\
                    + str(workshop_booking_id) + ')'
            
            list_of_workshop_enrollment.append((counter, workshop_booking_id, deID))
            counter += 1
            
            if counter % 900 == 899:
                q += '\n' + q_begin
                
            elif j < len(booked) - 1 or i < len(list_of_workshop_bookings) - 1:
                q += ', '
    return q


In [346]:
def days_enrolment():
    cols = '(Person_ID, Reservation_ID, Student)'
    q_begin = 'INSERT INTO [dbo].[Conference_Day_Participant] ' + cols + ' VALUES '
    q = q_begin
    counter = 1

    
    student_list = [x[1] for x in list_of_students]
    
    
    
    for i in range(0, len(list_of_conferences_bookings)):
        day_booking_id = i + 1
        
        if random.randint(0, 15) == 0:
            normal_participants_number = random.randint(0, list_of_conferences_bookings[i][2])
        else:
            normal_participants_number = list_of_conferences_bookings[i][2]
            
        if random.randint(0, 15) == 0:
            student_participants_number = random.randint(0, list_of_conferences_bookings[i][3])
        else:
            student_participants_number = list_of_conferences_bookings[i][3]
        
        nparticipants_list = random.sample(range(1, len(list_of_people)+1),
                                          min(normal_participants_number, len(list_of_people)))
        
        sparticipants_list = random.sample(range(1, len(student_list)+1),
                                          min(student_participants_number, len(student_list)))
        
   
        participantsList = [[x,0] for x in nparticipants_list]
        studentList = [[x,1] for x in sparticipants_list]
        
        if len(studentList) != 0 :
            participantsList += studentList
        
        for j, participantID in enumerate(participantsList):
            
            q = q + '(' + str(participantID[0]) + ', '\
                        + str(day_booking_id) + ', '\
                        + str(participantID[1]) + ')'
            list_of_days_enrollment.append((counter, participantID[0], day_booking_id, participantID[1]))
            
            counter += 1
            if counter%900 == 899:
                q += '\n' + q_begin
            elif j < len(participantsList) - 1:
                q += ', '
        
        
        if i < len(list_of_days_bookings) - 1 and participantsList:
            q += ', '
    return q

In [374]:
import math


def workshops_bookings():
    cols = '(Reservation_ID, Workshop_ID, Conference_Day_ID, Ticket_Count,Is_Cancelled)'
    q = 'INSERT INTO [dbo].[Workshop_Reservation] ' + cols + ' VALUES '
    counter = 1
    for i, workshop in enumerate(list_of_workshops_days):
     
        workshop_id = workshop[0]
        day_id = workshop[1]
        
        if random.randint(0, 20) == 0:
            places = random.randint(1, workshop[2])
        else:
            places = workshop[2]
            
        booked_for_day = []
        
        for day_booking in list_of_conferences_bookings:
            if day_booking[6] == day_id:
                booked_for_day += [day_booking]
                

                
        bookingLen = len(booked_for_day)

                

        for j,dayBooking in enumerate(booked_for_day):
            if (places != 0):
                is_cancelled = list_of_conferences_bookings[dayBooking[0] - 1][7]
                if is_cancelled == 0 and random.randint(0, 10) == 0:
                    is_cancelled = 1

                ticket_count = min(places, day_booking[2] + day_booking[3], math.ceil(workshop[2]/bookingLen))

                places -= ticket_count
                
                q = q + '(' + str(dayBooking[0]) + ', '\
                            + str(workshop_id) + ', '\
                            + str(day_id) + ', '\
                            + str(ticket_count) + ', '\
                            + str(is_cancelled) + ')'

                list_of_workshop_bookings.append((counter, dayBooking[0], workshop_id, day_id, ticket_count, is_cancelled))
                counter += 1
                if j < len(booked_for_day) - 1 or i < len(list_of_workshops):
                    q += ', '
            
            
    return q



In [246]:
def sum_price(booking_ID):
    booking = list_of_conferences_bookings[booking_ID-1]
    normal_tickets = booking[2]
    student_tickets = booking[3]
    conference_day = list_of_days[booking[6]-1]
    conference = list_of_conferences[conference_day[1]-1]
    
    total_price = normal_tickets * conference_day[4] + student_tickets * conference_day[4] * ((100 - conference[5])/100)
    
    max_discount = 0
    
    for x in list_of_discounts :
        if (x[3] == conference_day[0] ):
            if x[1] > max_discount and ( booking[1] + datetime.timedelta(days=(x[2])) ) < conference_day[2]:
                max_discount = x[1]
    
    return total_price * ((100 - max_discount)/100)
                 

In [433]:

def payments():
    cols = '(Payment_Date, Amount_Paid, Reservation_ID)'
    q_begin = 'INSERT INTO [dbo].[Payments] ' + cols + ' VALUES '
    q = q_begin
    for i, booking in enumerate(list_of_conferences_bookings):
        
        conference_booking_id = booking[0]
        to_pay = sum_price(conference_booking_id)
        
        conference_day_id = booking[6]
        register_date = booking[1]
        
        conference_day = list_of_days[conference_day_id-1]
        start_date = conference_day[2]
        day_diff = (start_date - register_date).days
        
        payment_date = register_date + datetime.timedelta(days=random.randint(0, day_diff))
        number_of_payments = random.choice([0, 0, 1, 1, 1, 2, 5, 7])
        
        for j in range(0, number_of_payments):
            if j == number_of_payments - 1:
                if random.randint(0, 10) == 0 and to_pay > 0.1:
                    value = to_pay/(random.randint(2, 7))
                elif random.randint(0, 30):
                    value = to_pay * 2.4
                else:
                    value = to_pay
            else:
                if random.randint(0, 30) == 0:
                    value = -random.uniform(1, to_pay)
                else:
                    value = random.uniform(1, to_pay)
                to_pay -= value
            q = q + '(' + str(payment_date) + ', \''\
                        + str(value) + '\', '\
                        + str(conference_booking_id) + ')'
            list_of_payments.append((i + 1, conference_booking_id, payment_date, value))
            if i % 900 == 899:
                q += '\n' + q_begin
            elif i < len(list_of_conferences_bookings) - 1 or j < number_of_payments - 1:
                q += ', '
    return q

In [233]:

def conferences_bookings():
    cols = '(Reservation_Date, Normal_Ticket_Count, Student_Ticket_Count,Amount_To_Pay ,Client_ID, Conference_Day_ID, isCancelled)'
    q = 'INSERT INTO [dbo].[Reservation] ' + cols + ' VALUES '
    
    counter = 1
    for i, conference_day in enumerate(list_of_days):
        conference_day_id = conference_day[0]
        
        participants_limit = conference_day[3]
        
        client_number = min(participants_limit//3, len(list_of_clients))
        
        clients_list = random.sample(range(1, len(list_of_clients)+1),
                                       client_number)
        
        for j, customerID in enumerate(clients_list):
            
            register_date = list_of_days[conference_day_id - 1][2] - datetime.timedelta(days=random.randint(1, 60))
            
            if random.randint(0, 30) == 0 or conference_day[5]:
                cancelled = 1
            else:
                cancelled = 0
                
            normal = min(random.randint(0,5), participants_limit)
            student = min(random.randint(0,5), participants_limit)
            
            if (normal == 0 and student == 0):
                normal = random.randint(1,3)
            
            if (cancelled == 0):
                participants_limit - normal - student
            
            amount_to_pay = 0
            
            if (j == len(clients_list)-1 ):
                normal += int (participants_limit/2);
                participants_limit -= int(participants_limit/2)
                student += participants_limit
                
            q = q + '(' + str(register_date) + ', '\
                        + str(normal) + ', \''\
                        + str(student) + '\', '\
                        + str(amount_to_pay) + '\', '\
                        + str(customerID) + '\', '\
                        + str(conference_day_id) + '\', '\
                        + str(cancelled) + ')'
            
            list_of_conferences_bookings.append((counter, register_date, normal, student, amount_to_pay,customerID, conference_day_id,cancelled))
            counter += 1
            if j < len(clients_list) - 1:
                q += ', '
        if i < len(list_of_conferences) - 1 and clients_list:
            q += ', '
    return q


In [216]:

def workshops_in_day():
    cols = '(Workshop_ID, Conference_Day_ID, Participants_Limit, Price, Room, From, To, isCancelled)'
    q = 'INSERT INTO [dbo].[Workshops] ' + cols + ' VALUES '
    counter = 1
    for i, day in enumerate(list_of_days):
        day_id = day[0]
        conference_id = day[1]
        workshops_number = random.randint(3, 5)
        for j in range(1, workshops_number):
        
            participants_limit = random.randint(10,26)
            
            room = str(random.randint(100,300)) + random.choice(('A','B','C','D')) 
            start_hour = random.randint(8, 19)
            start_time = str(start_hour) + ":" + str(random.choice(('00', '30', '15')))
            end_hour = start_hour + random.choice((1, 2, 3, 4))
            end_time = str(end_hour) + ":" + str(random.choice(('00', '30', '15')))
            price = random.randint(3, 15)*10
            
            is_cancelled = 0
            if (random.randint(0,26) == 10):
                is_cancelled = 1
            
            workshop_ID = random.randint(1, len(list_of_workshops) )
            
            q = q + '(' + str(workshop_ID) + ', \''\
                        + str(conference_id) + '\', '\
                        + str(participants_limit) + ', \''\
                        + str(price) + '\', \''\
                        + str(room) + '\', \''\
                        + str(start_time) + '\', \''\
                        + str(end_time) + '\', '\
                        + str(is_cancelled) + ')'
            
            list_of_workshops_days.append((counter, workshop_ID, conference_id, participants_limit, price, room, start_time, end_time, is_cancelled))
            counter += 1
            if j < workshops_number - 1 or i < len(list_of_days):
                q += ', '
    return q


In [202]:
def discount():
    cols = '(Discount_Percentage, Days_Before_Conference, Conference_Day_ID)'
    q = 'INSERT INTO [dbo].[Discount] ' + cols + ' VALUES '
    counter = 1
    for i, conference_day in enumerate(list_of_days):
        conference_day_ID = list_of_days[i][0]
        discount_thresholds = random.randint(2,4)
        discount_initial_value = random.randint(15,45)
        initial_days_before_conference = random.randint(20,120)
        for j in range(1,discount_thresholds + 1):
            percentage = discount_initial_value/j
            days_before = initial_days_before_conference/j
            q = q + '(' + str(percentage) + ', '\
                        + str(days_before) + ', '\
                        + str(conference_day_ID) + ')'
            list_of_discounts.append((counter, percentage,
                                 days_before, conference_day_ID))
            counter += 1
            if j < discount_thresholds :
                q += ', '
    return q
    

In [186]:

def days():
    cols = '(Conference_ID, Date, Participants_Limit, Base_Price, Is_Cancelled)'
    q = 'INSERT INTO [dbo].[Conference_Day] ' + cols + ' VALUES '
    counter = 1
    for i, conference in enumerate(list_of_conferences):
        conference_id = conference[0]
        number_of_days_in_conference = abs((conference[3] - conference[2]).days)
        for j in range(0, number_of_days_in_conference + 1):
            is_cancelled = 0
            if (random.randint(0,20) == 10):
                is_cancelled = 1
            date = conference[2] + datetime.timedelta(days=j)
            participants_limit = random.randint(50,110)
            base_price = random.randint(12,40) * 10
            q = q + '(' + str(conference_id) + ', '\
                        + str(date) + ', '\
                        + str(participants_limit) + ', '\
                        + str(base_price) + ', '\
                        + str(is_cancelled) + ')'
            list_of_days.append((counter, conference_id,
                                 date, participants_limit, base_price, is_cancelled))
            counter += 1
            if j < number_of_days_in_conference or i < len(list_of_conferences) - 1:
                q += ', '
    return q

In [182]:

def conferences(n):
    cols = '(Conference_Name, Start_Date, End_Date, Place_ID, Student_Discount, is_Cancelled)'
    q = 'INSERT INTO [dbo].[Conference] ' + cols + ' VALUES '
    for i in range(0, n):
        start_date = datetime.date(2017, 1, 1) + datetime.timedelta(days=random.randint(0, 1095))
        end_date = start_date + datetime.timedelta(days=random.randint(1, 3))
        conference_name = random.choice(('Driving', 'History of', 'Collecting', 'Colors of', '50 shades of')) \
            + ' ' + random.choice(('Cars', 'USA', 'Stones', 'Autumn', 'Mike'))
        student_discount = random.randint(0, 100)/100
        place_ID = list_of_conferencesPlaces[random.randint(0,len(list_of_conferencesPlaces)-1)][0]
        if random.randint(0, 20) == 0:
            is_cancelled = 1
        else:
            is_cancelled = 0
        q = q + '(\'' + conference_name + '\', \''\
                + str(start_date) + '\', \''\
                + str(end_date) + '\', ' \
                + str(place_ID) + '\', ' \
                + str(student_discount) + ', ' \
                + str(is_cancelled) + ')'
        list_of_conferences.append((i + 1, conference_name, start_date, end_date,place_ID, student_discount, is_cancelled))
        if i % 900 == 899:
            q += '\n' + q_begin
        if i < n-1:
            q += ', '
    return q


In [178]:

def conferencePlaces(n):
    cols = '(Country, City, Street, Postal_Code)'
    q = 'INSERT INTO [dbo].[Conference_Place] ' + cols + ' VALUES '
    for i in range(0, n):
        city = data['city'][i]
        country = data['country'][i]
        psCode = data['postal_code'][i]
        address = data['address'][i]
        q = q + '(\'' + country + '\', \''\
                + city + '\', \''\
                + address + '\', ' \
                + psCode + ')'
        list_of_conferencesPlaces.append((i + 1, country, city, address, psCode))
        if i < n-1:
            q += ', '
    return q

In [167]:
def workshops(n):
    cols = '(Subject, Description)'
    q = 'INSERT INTO [dbo].[Workshop] ' + cols + ' VALUES '
    subjects1 = ['Pierwsze', 'Drugie', 'Trzecie', 'Czwarte','Piąte']
    subjects2 = ['pojawienie','wystapienie','zjedzenie','zbudowanie','ugotowanie']
    subjects3 = ['czlowieka','budynku','kosmosu','kotka','piasku']
    for i in range(0, n):
        subject = subjects1[random.randint(0,4)] + ' ' + subjects2[random.randint(0,4)] + ' ' + subjects3[random.randint(0,4)] 
        description = "toDo " + str(i)
        q = q + '(\'' + subject + '\', \''\
                + description + ')'
        list_of_workshops.append((i + 1, subject, description))
        if i % 900 == 899:
            q += '\n' + q_begin
        if i < n-1:
            q += ', '
    return q

In [196]:
def student():
    cols = '(Person_ID, University, Faculty, is_valid)'
    q= 'INSERT INTO [dbo].[Student] ' + cols + ' VALUES '
    Universities = ['AGH','UWR','UJK','Stanford','MIT','UJ']
    Faculty = ["informatyka","medycyna","prawo"]
    for i in range(0, int (len(list_of_people)/8)):
        university = Universities[random.randint(0,len(Universities)-1)]
        faculty = Faculty[random.randint(0,len(Faculty)-1)]
        person_id = list_of_people[i*2][0]
        q = q + '(\'' + str(person_id) + '\', \''\
                    + university + '\', \''\
                    + faculty + '\', '\
                    + str(1) + ')'
        list_of_students.append((i+1, person_id, university, faculty, str(1)))
        if i % 900 == 899:
            q += '\n' + q_begin
        if i < len(list_of_people)/8 :
            q += ', '
    return q 

In [129]:

def company(n):
    cols = '(Company_Name,Client_ID, Address, City, Country, Phone, Mail, Contact_Person)'
    q= 'INSERT INTO [dbo].[Company] ' + cols + ' VALUES '
    for i in range(0, n):
        name = data['companies'][i]
        adress = data['address'][i+7000]
        city = data['city'][i+7000]
        country = data['country'][i+7000]
        phone = data['phone'][i+7000]
        mail = data['mail'][i+7000]
        contact_person = data['first_name'][i] + data['surname'][i]
        popIndex = len(list_of_clients_to_destroy)
        client_ID = list_of_clients_to_destroy.pop(random.randint(0, popIndex - 1 ))
        popIndex -= 1
        q = q + '(\'' + name + '\', \''\
                    + str(client_ID) + '\', \''\
                    + adress + '\', '\
                    + city + '\', '\
                    + country + '\', '\
                    + phone + '\', '\
                    + mail + '\', '\
                    + contact_person + ')'
        list_of_companies.append((i+1, name, client_ID, adress, city, country, phone, mail,contact_person))
        if i % 900 == 899:
            q += '\n' + q_begin
        if i < n-1:
            q += ', '
    return q


In [130]:

def people(n):
    cols = '(Client_ID,First_Name, Last_Name, Address,City, Country ,Phone, Mail)'
    q= 'INSERT INTO [dbo].[Person] ' + cols + ' VALUES '
    for i in range(0, n):
        #print(data['first_name'])
        first_name = data['first_name'][i]
        last_name = data['surname'][i]
        adress = data['address'][i]
        city = data['city'][i]
        country = data['country'][i]
        phone = data['phone'][i]
        mail = data['mail'][i]
        popIndex = len(list_of_clients_to_destroy)
        client_ID = list_of_clients_to_destroy.pop(random.randint(0, popIndex-1))
        popIndex -= 1
        q = q + '(\'' + str(client_ID) + '\', \''\
                    + first_name + '\', \''\
                    + last_name + '\', \''\
                    + adress + '\', '\
                    + city + '\', '\
                    + country + '\', '\
                    + str(phone) + '\', '\
                    + str(mail) + ')'
        list_of_people.append((i+1, first_name, last_name, adress, city, country, phone, mail))
        if i % 900 == 899:
            q += '\n' + q_begin
        if i < n-1:
            q += ', '
    return q



In [131]:
def clients(n):
    q= '''INSERT INTO [dbo].[Client] DEFAULT VALUES'''
    for i in range(0, n):
        list_of_clients.append(i+1)
        list_of_clients_to_destroy.append(i+1)
        q += '''
        INSERT INTO [dbo].[Client] DEFAULT VALUES'''

    return q

In [406]:
print(clients(25))


INSERT INTO [dbo].[Client] DEFAULT VALUES
        INSERT INTO [dbo].[Client] DEFAULT VALUES
        INSERT INTO [dbo].[Client] DEFAULT VALUES
        INSERT INTO [dbo].[Client] DEFAULT VALUES
        INSERT INTO [dbo].[Client] DEFAULT VALUES
        INSERT INTO [dbo].[Client] DEFAULT VALUES
        INSERT INTO [dbo].[Client] DEFAULT VALUES
        INSERT INTO [dbo].[Client] DEFAULT VALUES
        INSERT INTO [dbo].[Client] DEFAULT VALUES
        INSERT INTO [dbo].[Client] DEFAULT VALUES
        INSERT INTO [dbo].[Client] DEFAULT VALUES
        INSERT INTO [dbo].[Client] DEFAULT VALUES
        INSERT INTO [dbo].[Client] DEFAULT VALUES
        INSERT INTO [dbo].[Client] DEFAULT VALUES
        INSERT INTO [dbo].[Client] DEFAULT VALUES
        INSERT INTO [dbo].[Client] DEFAULT VALUES
        INSERT INTO [dbo].[Client] DEFAULT VALUES
        INSERT INTO [dbo].[Client] DEFAULT VALUES
        INSERT INTO [dbo].[Client] DEFAULT VALUES
        INSERT INTO [dbo].[Client] DEFAULT VALUES
        

In [407]:
print(people(20))

INSERT INTO [dbo].[Person] (Client_ID,First_Name, Last_Name, Address,City, Country ,Phone, Mail) VALUES ('23', 'Sharon', 'Webster', '2914 Pinchelone Street', Newport News', United States', 757-380-3307', SharonTWebster@superrito.com), ('15', 'Dawn', 'Espinoza', '4148 Wood Duck Drive', Iron Mountain', United States', 906-369-5142', DawnMEspinoza@gustr.com), ('14', 'Haydee', 'Clark', '1724 Bird Spring Lane', Sugar Land', United States', 281-527-7519', HaydeeDClark@teleworm.us), ('19', 'Samuel', 'Williams', '4680 Hickory Lane', Washington', United States', 202-508-5305', SamuelEWilliams@einrot.com), ('4', 'Marva', 'Zubia', '52 Murry Street', Virginia Beach', United States', 757-491-6703', MarvaRZubia@fleckens.hu), ('6', 'Laureen', 'Twomey', '4672 Round Table Drive', Cincinnati', United States', 513-786-6439', LaureenRTwomey@cuvox.de), ('5', 'Marjorie', 'Mouton', '2197 Godfrey Road', New York', United States', 212-587-8166', MarjorieWMouton@jourrapide.com), ('20', 'Roderick', 'Webre', '324

In [408]:
print(list_of_clients_to_destroy)

[3, 12, 17, 22, 25]


In [409]:
print(company(5))

INSERT INTO [dbo].[Company] (Company_Name,Client_ID, Address, City, Country, Phone, Mail, Contact_Person) VALUES ('Trak Auto', '22', '2390 Frum Street', Spring Hill', United States', 615-302-7438', NancyRStuckey@einrot.com', SharonWebster), ('Parts America', '12', '4577 Angus Road', New York', United States', 212-425-4999', EdwardEChase@rhyta.com', DawnEspinoza), ('Desert Garden Help', '3', '2088 Berry Street', Colorado Springs', United States', 719-577-6479', NicoleJJohnson@gustr.com', HaydeeClark), ('Four Leaf Clover', '25', '875 Copperhead Road', Meriden', United States', 860-671-6786', JosephJVogt@jourrapide.com', SamuelWilliams), ('Super Shops', '17', '2226 Columbia Boulevard', Baltimore', United States', 410-396-7393', MaryMMacklin@dayrep.com', MarvaZubia)


In [410]:
print(student())

INSERT INTO [dbo].[Student] (Person_ID, University, Faculty, is_valid) VALUES ('1', 'UJK', 'informatyka', 1), ('3', 'UJ', 'medycyna', 1), 


In [411]:
print(workshops(8))

INSERT INTO [dbo].[Workshop] (Subject, Description) VALUES ('Drugie pojawienie piasku', 'toDo 0), ('Piąte ugotowanie piasku', 'toDo 1), ('Drugie zjedzenie piasku', 'toDo 2), ('Czwarte wystapienie budynku', 'toDo 3), ('Trzecie zjedzenie kosmosu', 'toDo 4), ('Trzecie ugotowanie budynku', 'toDo 5), ('Trzecie zbudowanie kosmosu', 'toDo 6), ('Pierwsze zbudowanie budynku', 'toDo 7)


In [412]:
print(conferencePlaces(6))

INSERT INTO [dbo].[Conference_Place] (Country, City, Street, Postal_Code) VALUES ('United States', 'Newport News', '2914 Pinchelone Street', 23607), ('United States', 'Iron Mountain', '4148 Wood Duck Drive', 49801), ('United States', 'Sugar Land', '1724 Bird Spring Lane', 77478), ('United States', 'Washington', '4680 Hickory Lane', 20005), ('United States', 'Virginia Beach', '52 Murry Street', 23451), ('United States', 'Cincinnati', '4672 Round Table Drive', 45241)


In [413]:
print(conferences(4))

INSERT INTO [dbo].[Conference] (Conference_Name, Start_Date, End_Date, Place_ID, Student_Discount, is_Cancelled) VALUES ('History of USA', '2019-01-29', '2019-01-30', 1', 0.65, 0), ('History of Mike', '2019-09-09', '2019-09-12', 3', 0.25, 0), ('History of USA', '2017-10-15', '2017-10-17', 5', 0.31, 0), ('Colors of Cars', '2017-08-07', '2017-08-08', 1', 0.52, 0)


In [414]:
print(days())

INSERT INTO [dbo].[Conference_Day] (Conference_ID, Date, Participants_Limit, Base_Price, Is_Cancelled) VALUES (1, 2019-01-29, 97, 330, 0), (1, 2019-01-30, 74, 150, 0), (2, 2019-09-09, 94, 220, 0), (2, 2019-09-10, 70, 380, 0), (2, 2019-09-11, 75, 200, 0), (2, 2019-09-12, 70, 370, 0), (3, 2017-10-15, 98, 290, 1), (3, 2017-10-16, 70, 180, 0), (3, 2017-10-17, 91, 150, 0), (4, 2017-08-07, 75, 310, 0), (4, 2017-08-08, 79, 150, 0)


In [415]:
print(discount())

INSERT INTO [dbo].[Discount] (Discount_Percentage, Days_Before_Conference, Conference_Day_ID) VALUES (18.0, 36.0, 1), (9.0, 18.0, 1)(20.0, 31.0, 2), (10.0, 15.5, 2)(37.0, 104.0, 3), (18.5, 52.0, 3), (12.333333333333334, 34.666666666666664, 3), (9.25, 26.0, 3)(42.0, 106.0, 4), (21.0, 53.0, 4), (14.0, 35.333333333333336, 4)(34.0, 35.0, 5), (17.0, 17.5, 5)(44.0, 85.0, 6), (22.0, 42.5, 6), (14.666666666666666, 28.333333333333332, 6), (11.0, 21.25, 6)(31.0, 100.0, 7), (15.5, 50.0, 7), (10.333333333333334, 33.333333333333336, 7), (7.75, 25.0, 7)(15.0, 56.0, 8), (7.5, 28.0, 8), (5.0, 18.666666666666668, 8)(45.0, 33.0, 9), (22.5, 16.5, 9), (15.0, 11.0, 9), (11.25, 8.25, 9)(29.0, 38.0, 10), (14.5, 19.0, 10)(43.0, 85.0, 11), (21.5, 42.5, 11), (14.333333333333334, 28.333333333333332, 11), (10.75, 21.25, 11)


In [416]:
print(workshops_in_day())

INSERT INTO [dbo].[Workshops] (Workshop_ID, Conference_Day_ID, Participants_Limit, Price, Room, From, To, isCancelled) VALUES (6, '1', 22, '130', '267D', '13:30', '14:15', 0), (4, '1', 16, '130', '129D', '16:30', '19:15', 0), (6, '1', 10, '30', '212A', '13:00', '14:15', 0), (1, '1', 19, '50', '268D', '18:00', '19:00', 0), (1, '1', 22, '70', '251C', '10:00', '11:15', 0), (7, '1', 22, '120', '182C', '17:15', '21:15', 0), (7, '2', 16, '40', '244B', '12:15', '14:15', 0), (5, '2', 22, '110', '129A', '14:00', '15:15', 0), (8, '2', 20, '80', '287D', '14:15', '17:30', 0), (1, '2', 12, '100', '239B', '16:00', '20:00', 0), (8, '2', 24, '130', '107A', '13:30', '15:00', 0), (1, '2', 13, '50', '193B', '14:15', '15:30', 0), (1, '2', 10, '40', '180B', '10:15', '14:30', 1), (4, '2', 19, '130', '215C', '11:00', '14:30', 0), (7, '2', 18, '80', '133B', '19:30', '22:00', 0), (8, '2', 14, '60', '217B', '16:15', '19:00', 0), (4, '2', 11, '60', '109A', '12:15', '15:30', 0), (3, '2', 25, '80', '251C', '13:15'

In [417]:
print(conferences_bookings())

INSERT INTO [dbo].[Reservation] (Reservation_Date, Normal_Ticket_Count, Student_Ticket_Count,Amount_To_Pay ,Client_ID, Conference_Day_ID, isCancelled) VALUES (2019-01-11, 5, '0', 0', 13', 1', 0), (2019-01-28, 5, '5', 0', 22', 1', 0), (2019-01-07, 1, '1', 0', 3', 1', 0), (2018-11-30, 4, '3', 0', 18', 1', 0), (2019-01-16, 1, '2', 0', 17', 1', 0), (2019-01-10, 1, '4', 0', 8', 1', 0), (2018-12-05, 3, '5', 0', 19', 1', 0), (2019-01-23, 0, '1', 0', 24', 1', 0), (2018-12-29, 5, '5', 0', 16', 1', 0), (2018-12-06, 1, '4', 0', 25', 1', 0), (2018-12-23, 2, '1', 0', 7', 1', 0), (2019-01-02, 2, '5', 0', 6', 1', 0), (2018-12-22, 2, '3', 0', 4', 1', 0), (2019-01-18, 0, '2', 0', 1', 1', 0), (2019-01-01, 3, '2', 0', 5', 1', 0), (2019-01-23, 1, '5', 0', 14', 1', 0), (2018-12-11, 0, '2', 0', 23', 1', 0), (2018-12-30, 5, '4', 0', 12', 1', 0), (2018-12-08, 2, '2', 0', 10', 1', 0), (2018-12-23, 5, '3', 0', 9', 1', 0), (2018-12-09, 4, '1', 0', 2', 1', 0), (2018-12-10, 1, '0', 0', 20', 1', 0), (2019-01-10, 3,

In [432]:
print(payments())

[(1, datetime.date(2019, 1, 11), 5, 0, 0, 13, 1, 0), (2, datetime.date(2019, 1, 28), 5, 5, 0, 22, 1, 0), (3, datetime.date(2019, 1, 7), 1, 1, 0, 3, 1, 0), (4, datetime.date(2018, 11, 30), 4, 3, 0, 18, 1, 0), (5, datetime.date(2019, 1, 16), 1, 2, 0, 17, 1, 0), (6, datetime.date(2019, 1, 10), 1, 4, 0, 8, 1, 0), (7, datetime.date(2018, 12, 5), 3, 5, 0, 19, 1, 0), (8, datetime.date(2019, 1, 23), 0, 1, 0, 24, 1, 0), (9, datetime.date(2018, 12, 29), 5, 5, 0, 16, 1, 0), (10, datetime.date(2018, 12, 6), 1, 4, 0, 25, 1, 0), (11, datetime.date(2018, 12, 23), 2, 1, 0, 7, 1, 0), (12, datetime.date(2019, 1, 2), 2, 5, 0, 6, 1, 0), (13, datetime.date(2018, 12, 22), 2, 3, 0, 4, 1, 0), (14, datetime.date(2019, 1, 18), 0, 2, 0, 1, 1, 0), (15, datetime.date(2019, 1, 1), 3, 2, 0, 5, 1, 0), (16, datetime.date(2019, 1, 23), 1, 5, 0, 14, 1, 0), (17, datetime.date(2018, 12, 11), 0, 2, 0, 23, 1, 0), (18, datetime.date(2018, 12, 30), 5, 4, 0, 12, 1, 0), (19, datetime.date(2018, 12, 8), 2, 2, 0, 10, 1, 0), (20, 

In [420]:
print(workshops_bookings())

INSERT INTO [dbo].[Workshop_Reservation] (Reservation_ID, Workshop_ID, Conference_Day_ID, Ticket_Count,Is_Cancelled) VALUES (123, 1, 6, 1, 0), (75, 2, 4, 1, 0), (123, 3, 6, 1, 0), (1, 4, 1, 1, 0), (1, 5, 1, 1, 0), (146, 6, 7, 1, 1), (146, 7, 7, 1, 1), (147, 7, 7, 1, 1), (98, 8, 5, 1, 1), (99, 8, 5, 1, 0), (171, 9, 8, 1, 0), (172, 9, 8, 1, 1), (1, 10, 1, 1, 0), (2, 10, 1, 1, 1), (171, 11, 8, 1, 0), (172, 11, 8, 1, 0), (1, 12, 1, 1, 0), (2, 12, 1, 1, 0), (1, 13, 1, 1, 1), (2, 13, 1, 1, 0), (75, 14, 4, 1, 0), (76, 14, 4, 1, 0), (146, 15, 7, 1, 1), (147, 15, 7, 1, 1), (171, 16, 8, 1, 0), (172, 16, 8, 1, 0), (75, 17, 4, 1, 0), (76, 17, 4, 1, 0), (50, 18, 3, 1, 0), (98, 19, 5, 1, 1), (99, 19, 5, 1, 0), (1, 20, 1, 1, 0), (2, 20, 1, 1, 0), (3, 20, 1, 1, 0), (146, 21, 7, 1, 1), (147, 21, 7, 1, 1), (148, 21, 7, 1, 1), (123, 22, 6, 1, 0), (124, 22, 6, 1, 0), (125, 22, 6, 1, 0), (1, 23, 1, 1, 0), (2, 23, 1, 1, 0), (3, 23, 1, 1, 0), (1, 24, 1, 1, 0), (2, 24, 1, 1, 0), (3, 24, 1, 1, 0), (75, 25, 4, 

In [421]:
print(days_enrolment())

INSERT INTO [dbo].[Conference_Day_Participant] (Person_ID, Reservation_ID, Student) VALUES (3, 1, 0), (19, 1, 0), (15, 1, 0), (5, 1, 0), (6, 1, 0)(1, 2, 0), (12, 2, 0), (13, 2, 0), (20, 2, 0), (3, 2, 0), (2, 2, 1), (1, 2, 1)(17, 3, 0), (1, 3, 1)(2, 4, 0), (5, 4, 0), (20, 4, 0), (11, 4, 0), (2, 4, 1), (1, 4, 1)(8, 5, 0), (1, 5, 1), (2, 5, 1)(20, 6, 0), (1, 6, 1), (2, 6, 1)(5, 7, 0), (6, 7, 0), (9, 7, 0), (1, 7, 1), (2, 7, 1)(1, 8, 1)(9, 9, 0), (20, 9, 0), (13, 9, 0), (18, 9, 0), (10, 9, 0), (2, 9, 1), (1, 9, 1)(14, 10, 0), (2, 10, 1), (1, 10, 1)(1, 11, 0), (12, 11, 0), (1, 11, 1)(11, 12, 0), (20, 12, 0), (1, 12, 1), (2, 12, 1)(11, 13, 0), (13, 13, 0), (2, 13, 1), (1, 13, 1)(2, 14, 1), (1, 14, 1)(16, 15, 0), (18, 15, 0), (14, 15, 0), (1, 15, 1), (2, 15, 1)(14, 16, 0), (2, 16, 1), (1, 16, 1)(1, 17, 1), (2, 17, 1)(13, 18, 0), (11, 18, 0), (7, 18, 0), (2, 18, 0), (5, 18, 0), (2, 18, 1), (1, 18, 1)(17, 19, 0), (12, 19, 0), (2, 19, 1), (1, 19, 1)(17, 20, 0), (13, 20, 0), (1, 20, 1), (2, 20, 1

In [422]:
print(list_of_days)

[(1, 1, datetime.date(2019, 1, 29), 97, 330, 0), (2, 1, datetime.date(2019, 1, 30), 74, 150, 0), (3, 2, datetime.date(2019, 9, 9), 94, 220, 0), (4, 2, datetime.date(2019, 9, 10), 70, 380, 0), (5, 2, datetime.date(2019, 9, 11), 75, 200, 0), (6, 2, datetime.date(2019, 9, 12), 70, 370, 0), (7, 3, datetime.date(2017, 10, 15), 98, 290, 1), (8, 3, datetime.date(2017, 10, 16), 70, 180, 0), (9, 3, datetime.date(2017, 10, 17), 91, 150, 0), (10, 4, datetime.date(2017, 8, 7), 75, 310, 0), (11, 4, datetime.date(2017, 8, 8), 79, 150, 0)]


In [423]:
print(list_of_workshop_bookings)

[(1, 123, 1, 6, 1, 0), (2, 75, 2, 4, 1, 0), (3, 123, 3, 6, 1, 0), (4, 1, 4, 1, 1, 0), (5, 1, 5, 1, 1, 0), (6, 146, 6, 7, 1, 1), (7, 146, 7, 7, 1, 1), (8, 147, 7, 7, 1, 1), (9, 98, 8, 5, 1, 1), (10, 99, 8, 5, 1, 0), (11, 171, 9, 8, 1, 0), (12, 172, 9, 8, 1, 1), (13, 1, 10, 1, 1, 0), (14, 2, 10, 1, 1, 1), (15, 171, 11, 8, 1, 0), (16, 172, 11, 8, 1, 0), (17, 1, 12, 1, 1, 0), (18, 2, 12, 1, 1, 0), (19, 1, 13, 1, 1, 1), (20, 2, 13, 1, 1, 0), (21, 75, 14, 4, 1, 0), (22, 76, 14, 4, 1, 0), (23, 146, 15, 7, 1, 1), (24, 147, 15, 7, 1, 1), (25, 171, 16, 8, 1, 0), (26, 172, 16, 8, 1, 0), (27, 75, 17, 4, 1, 0), (28, 76, 17, 4, 1, 0), (29, 50, 18, 3, 1, 0), (30, 98, 19, 5, 1, 1), (31, 99, 19, 5, 1, 0), (32, 1, 20, 1, 1, 0), (33, 2, 20, 1, 1, 0), (34, 3, 20, 1, 1, 0), (35, 146, 21, 7, 1, 1), (36, 147, 21, 7, 1, 1), (37, 148, 21, 7, 1, 1), (38, 123, 22, 6, 1, 0), (39, 124, 22, 6, 1, 0), (40, 125, 22, 6, 1, 0), (41, 1, 23, 1, 1, 0), (42, 2, 23, 1, 1, 0), (43, 3, 23, 1, 1, 0), (44, 1, 24, 1, 1, 0), (45,

In [431]:
print(workshops_participation())

IndexError: list index out of range

In [266]:
print(list_of_days_bookings)

[]


In [1]:


print(participants(2900) + "\n" +
      customers(100) + "\n" +
      conferences(72) + "\n" +
      days() + "\n" +
      conferences_bookings() + "\n" +
      days_bookings() + "\n" +
      days_enrolment() + "\n" +
      prices() + "\n" +
      workshops() + "\n" +
      workshops_bookings() + "\n" +
      workshops_enrolment() + "\n" +
      payments() + "\n")

NameError: name 'participants' is not defined